In [1]:
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
file_name = 'data_256.pkl'
image_data, sentiment_data, sentence_data, sentence_tags, hashtag_data = pickle.load(open(file_name, 'rb'))

In [3]:
size = 3000

image_data = image_data[:size]
sentiment_data = sentiment_data[:size]
sentence_data = sentence_data[:size]
sentence_tags = sentence_tags[:size]
hashtag_data = hashtag_data[:size]

In [4]:
print(image_data.shape)
print(sentiment_data.shape)
print(sentence_data.shape)
print(len(sentence_tags))

(3000, 256, 256, 3)
(3000, 4)
(3000,)
3000


In [5]:
input_tags = [[tag[0] for tag in tags] for tags in sentence_tags]

In [6]:
from konlpy.tag import Komoran

komoran = Komoran()

output_tags = [[p[0] for p in komoran.pos(s)] for s in tqdm(sentence_data)]
[tags.insert(0, '\t') for tags in output_tags]
[tags.append('\n') for tags in output_tags]
None

100%|██████████| 3000/3000 [00:02<00:00, 1277.03it/s]


In [7]:
output_words = set()
for tags in output_tags:
    for tag in tags:
        if tag not in output_words:
            output_words.add(tag)
            
output_words = sorted(list(output_words))
output_words.insert(0, 'P')
n_output_words = len(output_words)

max_seq_length = max([len(_in) + len(_out) for _in, _out in zip(input_tags, output_tags)])

w2i_output = {w:i for i, w in enumerate(output_words)}
seq_list = []

for i, (_in, _out) in enumerate(zip(input_tags, output_tags)):
    for j in range(1, len(_out)):
        seq_list.append(_in + _out[:j+1])
        
n_seq_data = len(seq_list)

In [8]:
seq_data = np.zeros(shape=(n_seq_data, max_seq_length+1), dtype='int32')

for i, seq in enumerate(seq_list):
    seq_data[i, -len(seq):] = np.array([w2i_output[w] for w in seq])
    
x_seq_data = seq_data[:, :-1]

output_eye = np.eye(n_output_words)
y_seq_data = output_eye[seq_data[:, -1]]
# y_seq_data = seq_data[:, -1]

In [9]:
x_seq_data.shape, y_seq_data.shape

((129470, 232), (129470, 9948))

In [10]:
valid_size = 0
train_size = n_seq_data - valid_size
batch_size = 7000
n_batch = train_size // batch_size
n_valid = valid_size // batch_size

In [11]:
x_train, y_train = x_seq_data[:train_size], y_seq_data[:train_size]
x_valid, y_valid = x_seq_data[train_size:], y_seq_data[train_size:]

In [12]:
x_train.shape, y_train.shape

((129470, 232), (129470, 9948))

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, LSTM

In [14]:
model = Sequential()
model.add(Embedding(n_output_words, 64, input_length=max_seq_length))
model.add(LSTM(128))
model.add(Dense(n_output_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 232, 64)           636672    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 9948)              1283292   
Total params: 2,018,780
Trainable params: 2,018,780
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.fit(x_train, y_train, 
          validation_split=0.1,
          batch_size=batch_size,
          epochs=1000)

Epoch 1/1000
17/17 [==============================] - 7s 416ms/step - loss: 9.0088 - accuracy: 0.0304 - val_loss: 8.2438 - val_accuracy: 0.0385
Epoch 2/1000
17/17 [==============================] - 6s 372ms/step - loss: 7.3952 - accuracy: 0.0364 - val_loss: 6.9628 - val_accuracy: 0.0385
Epoch 3/1000
17/17 [==============================] - 6s 373ms/step - loss: 6.8286 - accuracy: 0.0364 - val_loss: 6.9727 - val_accuracy: 0.0385
Epoch 4/1000
17/17 [==============================] - 6s 363ms/step - loss: 6.7930 - accuracy: 0.0364 - val_loss: 6.9763 - val_accuracy: 0.0385
Epoch 5/1000
17/17 [==============================] - 6s 366ms/step - loss: 6.7831 - accuracy: 0.0364 - val_loss: 6.9849 - val_accuracy: 0.0385
Epoch 6/1000
17/17 [==============================] - 6s 367ms/step - loss: 6.7802 - accuracy: 0.0364 - val_loss: 6.9901 - val_accuracy: 0.0385
Epoch 7/1000
17/17 [==============================] - 6s 370ms/step - loss: 6.7786 - accuracy: 0.0364 - val_loss: 6.9985 - val_accuracy:

17/17 [==============================] - 6s 372ms/step - loss: 4.4652 - accuracy: 0.2500 - val_loss: 6.1261 - val_accuracy: 0.1698
Epoch 114/1000
17/17 [==============================] - 6s 371ms/step - loss: 4.4506 - accuracy: 0.2510 - val_loss: 6.1261 - val_accuracy: 0.1705
Epoch 115/1000
17/17 [==============================] - 6s 372ms/step - loss: 4.4360 - accuracy: 0.2524 - val_loss: 6.1280 - val_accuracy: 0.1695
Epoch 116/1000
17/17 [==============================] - 6s 374ms/step - loss: 4.4219 - accuracy: 0.2537 - val_loss: 6.1341 - val_accuracy: 0.1706
Epoch 117/1000
17/17 [==============================] - 6s 375ms/step - loss: 4.4074 - accuracy: 0.2545 - val_loss: 6.1332 - val_accuracy: 0.1703
Epoch 118/1000
17/17 [==============================] - 6s 372ms/step - loss: 4.3929 - accuracy: 0.2556 - val_loss: 6.1305 - val_accuracy: 0.1716
Epoch 119/1000
17/17 [==============================] - 6s 368ms/step - loss: 4.3783 - accuracy: 0.2571 - val_loss: 6.1334 - val_accuracy: 

17/17 [==============================] - 6s 367ms/step - loss: 3.2441 - accuracy: 0.3888 - val_loss: 6.4377 - val_accuracy: 0.1854
Epoch 226/1000
17/17 [==============================] - 6s 373ms/step - loss: 3.2360 - accuracy: 0.3909 - val_loss: 6.4405 - val_accuracy: 0.1845
Epoch 227/1000
17/17 [==============================] - 6s 370ms/step - loss: 3.2269 - accuracy: 0.3920 - val_loss: 6.4463 - val_accuracy: 0.1834
Epoch 228/1000
17/17 [==============================] - 6s 369ms/step - loss: 3.2191 - accuracy: 0.3931 - val_loss: 6.4478 - val_accuracy: 0.1834
Epoch 229/1000
17/17 [==============================] - 6s 372ms/step - loss: 3.2116 - accuracy: 0.3945 - val_loss: 6.4551 - val_accuracy: 0.1833
Epoch 230/1000
17/17 [==============================] - 6s 371ms/step - loss: 3.2037 - accuracy: 0.3954 - val_loss: 6.4615 - val_accuracy: 0.1823
Epoch 231/1000
17/17 [==============================] - 6s 373ms/step - loss: 3.1953 - accuracy: 0.3966 - val_loss: 6.4620 - val_accuracy: 

17/17 [==============================] - 6s 377ms/step - loss: 2.5264 - accuracy: 0.5021 - val_loss: 6.9583 - val_accuracy: 0.1860
Epoch 338/1000
17/17 [==============================] - 6s 372ms/step - loss: 2.5212 - accuracy: 0.5038 - val_loss: 6.9644 - val_accuracy: 0.1846
Epoch 339/1000
17/17 [==============================] - 6s 378ms/step - loss: 2.5159 - accuracy: 0.5042 - val_loss: 6.9651 - val_accuracy: 0.1848
Epoch 340/1000
17/17 [==============================] - 6s 373ms/step - loss: 2.5103 - accuracy: 0.5050 - val_loss: 6.9681 - val_accuracy: 0.1844
Epoch 341/1000
17/17 [==============================] - 6s 375ms/step - loss: 2.5059 - accuracy: 0.5058 - val_loss: 6.9737 - val_accuracy: 0.1844
Epoch 342/1000
17/17 [==============================] - 6s 375ms/step - loss: 2.4996 - accuracy: 0.5069 - val_loss: 6.9816 - val_accuracy: 0.1843
Epoch 343/1000
17/17 [==============================] - 6s 374ms/step - loss: 2.4964 - accuracy: 0.5073 - val_loss: 6.9912 - val_accuracy: 

17/17 [==============================] - 6s 375ms/step - loss: 2.0395 - accuracy: 0.5891 - val_loss: 7.5387 - val_accuracy: 0.1827
Epoch 450/1000
17/17 [==============================] - 6s 373ms/step - loss: 2.0367 - accuracy: 0.5897 - val_loss: 7.5459 - val_accuracy: 0.1808
Epoch 451/1000
17/17 [==============================] - 6s 372ms/step - loss: 2.0331 - accuracy: 0.5906 - val_loss: 7.5461 - val_accuracy: 0.1830
Epoch 452/1000
17/17 [==============================] - 6s 372ms/step - loss: 2.0298 - accuracy: 0.5911 - val_loss: 7.5535 - val_accuracy: 0.1828
Epoch 453/1000
17/17 [==============================] - 6s 375ms/step - loss: 2.0254 - accuracy: 0.5916 - val_loss: 7.5575 - val_accuracy: 0.1816
Epoch 454/1000
17/17 [==============================] - 6s 370ms/step - loss: 2.0234 - accuracy: 0.5923 - val_loss: 7.5495 - val_accuracy: 0.1820
Epoch 455/1000
17/17 [==============================] - 6s 368ms/step - loss: 2.0208 - accuracy: 0.5929 - val_loss: 7.5670 - val_accuracy: 

17/17 [==============================] - 6s 371ms/step - loss: 1.6787 - accuracy: 0.6590 - val_loss: 8.1674 - val_accuracy: 0.1762
Epoch 562/1000
17/17 [==============================] - 6s 373ms/step - loss: 1.6753 - accuracy: 0.6593 - val_loss: 8.1718 - val_accuracy: 0.1763
Epoch 563/1000
17/17 [==============================] - 6s 372ms/step - loss: 1.6720 - accuracy: 0.6606 - val_loss: 8.1773 - val_accuracy: 0.1763
Epoch 564/1000
17/17 [==============================] - 6s 375ms/step - loss: 1.6690 - accuracy: 0.6607 - val_loss: 8.1867 - val_accuracy: 0.1765
Epoch 565/1000
17/17 [==============================] - 6s 370ms/step - loss: 1.6672 - accuracy: 0.6616 - val_loss: 8.1803 - val_accuracy: 0.1766
Epoch 566/1000
17/17 [==============================] - 6s 375ms/step - loss: 1.6636 - accuracy: 0.6614 - val_loss: 8.1784 - val_accuracy: 0.1768
Epoch 567/1000
17/17 [==============================] - 6s 370ms/step - loss: 1.6627 - accuracy: 0.6619 - val_loss: 8.1992 - val_accuracy: 

17/17 [==============================] - 6s 373ms/step - loss: 1.4029 - accuracy: 0.7145 - val_loss: 8.8033 - val_accuracy: 0.1709
Epoch 674/1000
17/17 [==============================] - 6s 371ms/step - loss: 1.4003 - accuracy: 0.7149 - val_loss: 8.8212 - val_accuracy: 0.1717
Epoch 675/1000
17/17 [==============================] - 6s 374ms/step - loss: 1.4043 - accuracy: 0.7133 - val_loss: 8.8346 - val_accuracy: 0.1708
Epoch 676/1000
17/17 [==============================] - 6s 377ms/step - loss: 1.3971 - accuracy: 0.7153 - val_loss: 8.8260 - val_accuracy: 0.1712
Epoch 677/1000
17/17 [==============================] - 6s 371ms/step - loss: 1.3929 - accuracy: 0.7167 - val_loss: 8.8177 - val_accuracy: 0.1716
Epoch 678/1000
17/17 [==============================] - 6s 376ms/step - loss: 1.3903 - accuracy: 0.7167 - val_loss: 8.8309 - val_accuracy: 0.1712
Epoch 679/1000
17/17 [==============================] - 6s 373ms/step - loss: 1.3875 - accuracy: 0.7180 - val_loss: 8.8496 - val_accuracy: 

17/17 [==============================] - 6s 371ms/step - loss: 1.1831 - accuracy: 0.7600 - val_loss: 9.4604 - val_accuracy: 0.1645
Epoch 786/1000
17/17 [==============================] - 6s 373ms/step - loss: 1.1809 - accuracy: 0.7614 - val_loss: 9.4657 - val_accuracy: 0.1649
Epoch 787/1000
17/17 [==============================] - 6s 372ms/step - loss: 1.1792 - accuracy: 0.7609 - val_loss: 9.4605 - val_accuracy: 0.1651
Epoch 788/1000
17/17 [==============================] - 6s 376ms/step - loss: 1.1783 - accuracy: 0.7612 - val_loss: 9.4676 - val_accuracy: 0.1647
Epoch 789/1000
17/17 [==============================] - 6s 370ms/step - loss: 1.1757 - accuracy: 0.7623 - val_loss: 9.4686 - val_accuracy: 0.1664
Epoch 790/1000
17/17 [==============================] - 6s 377ms/step - loss: 1.1740 - accuracy: 0.7624 - val_loss: 9.4837 - val_accuracy: 0.1647
Epoch 791/1000
17/17 [==============================] - 6s 374ms/step - loss: 1.1722 - accuracy: 0.7626 - val_loss: 9.4943 - val_accuracy: 

17/17 [==============================] - 6s 373ms/step - loss: 1.0054 - accuracy: 0.7980 - val_loss: 10.0927 - val_accuracy: 0.1607
Epoch 898/1000
17/17 [==============================] - 6s 375ms/step - loss: 1.0043 - accuracy: 0.7973 - val_loss: 10.1040 - val_accuracy: 0.1594
Epoch 899/1000
17/17 [==============================] - 6s 373ms/step - loss: 1.0022 - accuracy: 0.7982 - val_loss: 10.0982 - val_accuracy: 0.1589
Epoch 900/1000
17/17 [==============================] - 6s 372ms/step - loss: 1.0003 - accuracy: 0.7984 - val_loss: 10.1004 - val_accuracy: 0.1593
Epoch 901/1000
17/17 [==============================] - 6s 375ms/step - loss: 1.0005 - accuracy: 0.7988 - val_loss: 10.1092 - val_accuracy: 0.1592
Epoch 902/1000
17/17 [==============================] - 6s 370ms/step - loss: 0.9985 - accuracy: 0.7994 - val_loss: 10.1233 - val_accuracy: 0.1600
Epoch 903/1000
17/17 [==============================] - 6s 374ms/step - loss: 0.9965 - accuracy: 0.7995 - val_loss: 10.1246 - val_acc

Epoch 953/1000
17/17 [==============================] - 6s 376ms/step - loss: 0.9307 - accuracy: 0.8140 - val_loss: 10.3975 - val_accuracy: 0.1573
Epoch 954/1000
17/17 [==============================] - 6s 375ms/step - loss: 0.9295 - accuracy: 0.8143 - val_loss: 10.4019 - val_accuracy: 0.1573
Epoch 955/1000
17/17 [==============================] - 6s 373ms/step - loss: 0.9276 - accuracy: 0.8144 - val_loss: 10.4135 - val_accuracy: 0.1565
Epoch 956/1000
17/17 [==============================] - 6s 377ms/step - loss: 0.9257 - accuracy: 0.8151 - val_loss: 10.4127 - val_accuracy: 0.1571
Epoch 957/1000
17/17 [==============================] - 6s 375ms/step - loss: 0.9240 - accuracy: 0.8152 - val_loss: 10.4207 - val_accuracy: 0.1572
Epoch 958/1000
17/17 [==============================] - 6s 373ms/step - loss: 0.9226 - accuracy: 0.8154 - val_loss: 10.4223 - val_accuracy: 0.1578
Epoch 959/1000
17/17 [==============================] - 6s 373ms/step - loss: 0.9216 - accuracy: 0.8157 - val_loss: 10

In [17]:
file_model = 'model_ver6.hdf5'
model.save(file_model)

In [13]:
from tensorflow.keras.models import load_model
file_model = 'model_ver6.hdf5'
model = load_model(file_model)

In [20]:
def sequence_generation(model, tags):
    tags = tags.copy()
    tags.append('\t')
    sentence = ''
    while tags[-1] != '\n' and len(tags) < max_seq_length:
        x_test = np.zeros(shape=(1, max_seq_length), dtype='int32')
        x_test[0, -len(tags):] = [w2i_output[w] for w in tags]
        result = model.predict_classes(x_test, verbose=0)
        tags.append(output_words[result[0]])
        sentence += ' ' + tags[-1]
    return sentence.replace('\n', '')

In [25]:
start_idx = 60
end_idx = start_idx + 20
for ori, tags in zip(sentence_data[start_idx:end_idx], input_tags[start_idx:end_idx]):
    print('원본')
    print(ori[1:-1])
    print(tags)
    print('예측')
    print(sequence_generation(model, tags))
    print()

원본
오버핏투버튼자켓 엉덩이를 덮는 기장의 오버핏투버튼자켓이예용 간절기부터 쌀쌀한 가을에도 애용하기 좋은 두께감 스타일리쉬한 핏과 세련된 컬러가 포인트예용 그레이 블랙
['오버핏투버튼자켓', '엉덩이', '덮', '기장', '오버핏투버튼자켓이예용', '간', '절기', '쌀쌀', '가을', '애용', '좋', '두께', '감', '스타일', '리', '쉬', '핏과', '세련', '컬러', '포인트', '예', '그레이', '블랙']
예측
 오버핏투버튼자켓 엉덩이 를 덮 는 기장 의 오버핏투버튼자켓이예용 간 절기 부터 쌀쌀 하 ㄴ 가을 에 도 애용 하 기 좋 은 두께 감 스타일 리 쉬 하 ㄴ 핏과 세련 되 ㄴ 컬러 가 포인트 예 용 그레이 블랙 

원본
데일리자켓은 도대체 안어울리는 옷이 뭐야 다 잘어울리고 난리 같이 매치하니 또 깔꾸미한 완스엉 넘나 예쁘죠
['데일리자켓은', '도대체', '안', '어울리', '옷', '뭐', '다', '잘', '어울리', '난리', '같이', '매치', '하니', '또', '깔', '꾸미', '하', '완스엉', '넘', '예쁘']
예측
 데일리자켓은 도대체 안 어울리 는 게 다임 제가 한 번 씩 중간 러 오 아서 먹 었 는데 아들 도 좋 아 용 

원본
벨뷰 사이즈 최상의 퀄리티 안전하고 빠른 배송 좋은 상품 약속합니다 내가 쓸 제품이라는 마음으로 확실한 제품 꼼꼼한 검수 약속합니다 저렴한 제품에는 이유가 있습니다 동일 퀄리티 대비 최저가 자부합니다 빠르지만 안전을 최우선으로 고객님이 확실하게 전달 받으실 수 있게 노력하겠습니다 게시된 사진은 모두 실사입니다 피드에 없는 제품도 문의주세요
['벨뷰', '사이즈', '최상', '퀄리티', '안전', '빠르', '배', '송', '좋', '상품', '약속', '내', '쓰', '제품', '이', '마음', '확실', '제품', '꼼꼼', '검', '수', '약속', '저렴', '제품', '이유', '있', '동일', '퀄리티', '대비', '최저', '자부', '빠르', '안

 사랑 가득 환하 ㄴ 웃음 이 설레임 가득 예쁘 었 던 정미 님 커플 이 감스에 방문 하 아 주 시 었 습니다 미소 에 서로 에 대하 ㄴ 사랑 이 시간 이 시간 이 었 습니다 

원본
추억을 담아드리는 사진관 소중한 시간을 사진으로 담아드립니다 흑백사진 평일 만원인기준 인화 액자 점 제공 수정컷 카톡메일 발송 주말의 경우 만원 추가 이벤트 참여시 전체이미지 제공
['추억', '담', '드리', '사진관', '소중', '시간', '사진', '담', '드리', '흑백', '사진', '평일', '만', '원인', '기준', '인화', '액자', '점', '제공', '수정', '컷', '카', '톡', '메이', '발송', '주말', '경우', '만원', '추가', '이벤트', '참여', '전체', '이미지', '제공']
예측
 추억 을 담 아 드리 는 사진관 소중 하 ㄴ 시간 을 사진 으로 담 아 드리 ㅂ니다 흑백 사진 평일 만 원인 기준 인화 액자 점 제공 수정 컷 카 톡 메이 ㄹ 발송 주말 의 경우 만원 추가 이벤트 참여 시 전체 이미지 제공 



['맛집', '오늘', '포인트', '이', '없', '거']

In [58]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [125]:
tf.reset_default_graph()

n_embedding = 64
n_hidden = 64
learning_rate = 1e-2

X = tf.placeholder(tf.int32, shape=(None, max_seq_length))
Y = tf.placeholder(tf.int32, shape=(None,))
# Y = tf.placeholder(tf.float32, shape=(None,))

embedding = tf.Variable(tf.random_uniform((n_output_words, n_embedding), -1, 1))
embed = tf.nn.embedding_lookup(embedding, X)

cell = tf.nn.rnn_cell.BasicLSTMCell(n_hidden)
outputs, states = tf.nn.dynamic_rnn(cell, embed, dtype=tf.float32)

model = tf.layers.dense(tf.reshape(outputs, [-1, max_seq_length*n_hidden]), n_output_words, activation=tf.nn.softmax)
# cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
cost = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

predict = tf.arg_max(model, 1)

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


In [126]:
with tf.device('/gpu:0'):
    sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True, 
                                            log_device_placement=True))
    sess.run(tf.global_variables_initializer())

Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:68:00.0, compute capability: 7.5



In [127]:
tf.trainable_variables()

[<tf.Variable 'Variable:0' shape=(6883, 64) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/kernel:0' shape=(128, 256) dtype=float32_ref>,
 <tf.Variable 'rnn/basic_lstm_cell/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense/kernel:0' shape=(14848, 6883) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(6883,) dtype=float32_ref>]

In [128]:
total_epoch = 100

# for epoch in tqdm(range(total_epoch)):
for epoch in range(total_epoch):
    total_loss = 0
    for batch in tqdm(range(n_batch)):
#     for batch in range(n_batch):
        batch_xs = x_train[batch*batch_size:(batch+1)*batch_size]
        batch_ys = y_train[batch*batch_size:(batch+1)*batch_size]
        
        _, loss = sess.run([optimizer, cost], feed_dict={
                X: batch_xs,
                Y: batch_ys
            })
        
        total_loss += loss
    
    total_valid_loss = 0
    for batch in range(n_valid):
        valid_xs = x_valid[batch*batch_size:(batch+1)*batch_size]
        valid_ys = y_valid[batch*batch_size:(batch+1)*batch_size]
        
        val_loss = sess.run(cost, feed_dict={
                X: valid_xs,
                Y: valid_ys
            })
        total_valid_loss += val_loss
    
    print('Epoch: %04d Cost: %.6f Valid Cost: %.6f' %(epoch + 1, total_loss / n_batch, total_valid_loss / n_valid))

  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0001 Cost: 8.803528 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0002 Cost: 8.800805 Valid Cost: 8.801259


  0%|          | 0/22 [00:00<?, ?it/s]

Epoch: 0003 Cost: 8.800805 Valid Cost: 8.801259


 45%|████▌     | 10/22 [00:02<00:03,  3.88it/s]


KeyboardInterrupt: 

In [115]:
def translate(tags):
    x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
    tags = tags + ['\t']
    idx = -len(tags)
    x_test[0, idx:] = [w2i_output[w] for w in tags]
    while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
        print(idx)
        pred = sess.run(predict, feed_dict={X: x_test})[0]
        x_test[0, idx-1:-1] = x_test[0, idx:]
        x_test[0, -1] = pred
        idx -= 1
    end = np.where(x_test[0]==w2i_output['\n'])[0]
    end = end[0] if end.shape[0] > 0 else -1
    ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
    return ''.join([output_words[ret]])

In [121]:
tags = input_tags[0]
x_test = np.zeros(shape=(1, max_seq_length), dtype='float32')
tags = tags + ['\t']
idx = -len(tags)
x_test[0, idx:] = [w2i_output[w] for w in tags]
while x_test[0, -1] != w2i_output['\n'] and x_test[0][0] == 0:
    print(idx)
    pred = sess.run(predict, feed_dict={X: x_test})[0]
    x_test[0, idx-1:-1] = x_test[0, idx:]
    x_test[0, -1] = pred
    idx -= 1
end = np.where(x_test[0]==w2i_output['\n'])[0]
end = end[0] if end.shape[0] > 0 else -1
ret = x_test[0, np.where(x_test[0]==w2i_output['\t'])[0][0]:end]
''.join([output_words[map(int, ret)]])

-7
-8
-9
-10
-11
-12
-13
-14
-15
-16
-17
-18
-19
-20
-21
-22
-23
-24
-25
-26
-27
-28
-29
-30
-31
-32
-33
-34
-35
-36
-37
-38
-39
-40
-41
-42
-43
-44
-45
-46
-47
-48
-49
-50
-51
-52
-53
-54
-55
-56
-57
-58
-59
-60
-61
-62
-63
-64
-65
-66
-67
-68
-69
-70
-71
-72
-73
-74
-75
-76
-77
-78
-79
-80
-81
-82
-83
-84
-85
-86
-87
-88
-89
-90
-91
-92
-93
-94
-95
-96
-97
-98
-99
-100
-101
-102
-103
-104
-105
-106
-107
-108
-109
-110
-111
-112
-113
-114
-115
-116
-117
-118


KeyboardInterrupt: 

In [124]:
''.join([output_words[int(i)] for i in ret])

'\t하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하하'